In [1]:
import sys
import numpy as np
import scipy as sp
import sklearn
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
#%pylab inline
%matplotlib notebook
#from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.utils.py3compat import annotate
from IPython.display import display
#from ipywidgets import FloatSlider
from ipywidgets import *

In [2]:
from InteractiveOptimisationAlgorithms.Functions import *
from InteractiveOptimisationAlgorithms.Matrix import *
from InteractiveOptimisationAlgorithms.Drawing import *
from InteractiveOptimisationAlgorithms.Algorithms import *
from InteractiveOptimisationAlgorithms.Constraints import *

In [3]:
EPSILON = 1E-6
ALPHA = 1
BETA = 0.5
GAMMA = 2
SIGMA = 0.5
PRINT = True

In [4]:
f1 = F1RosenbrockBananaFunction()
elements = np.array([[-1.9, 2]])
point = Matrix(1, 2, elements)
lower_bounds = [-100, -100]
upper_bounds = [100, 100]
implicit_constraint_1 = ImplicitConstraint1()
implicit_constraint_2 = ImplicitConstraint2()

implicit_constraints = [implicit_constraint_1,implicit_constraint_2]

box_algorithm = BoxAlgorithm(f1, lower_bounds, upper_bounds, implicit_constraints, EPSILON, ALPHA, PRINT)
solution_box, logger_box = box_algorithm.run(point)

In [5]:
min_X1 = -2
max_X1 = 3.5
min_X2 = -2
max_X2 = 2
number_of_samples_of_domain = 150

min_X1 = -5
max_X1 = 5
min_X2 = -5
max_X2 = 6
number_of_samples_of_domain = 150

In [6]:
X1_for_graph_before_meshgrid = np.linspace(min_X1, max_X1, number_of_samples_of_domain)
X2_for_graph_before_meshgrid = np.linspace(min_X2, max_X2, number_of_samples_of_domain)

X1_for_graph, X2_for_graph = np.meshgrid(X1_for_graph_before_meshgrid, X2_for_graph_before_meshgrid)
Z_for_graph = []
for x2 in X2_for_graph_before_meshgrid:
    Z = []
    for x1 in X1_for_graph_before_meshgrid:
        elements = np.array([[x1, x2]])
        matrix_x1_x2 = Matrix(1, 2, elements)
        Z.append(f1.valueAt(matrix_x1_x2))
    Z_for_graph.append(Z)

In [7]:
playMaxOfInterval = 0
X1_from_logger = []
X2_from_logger = []
Z_from_logger = []

for iteration in logger_box.getIterations():
    playMaxOfInterval = playMaxOfInterval + 1
    X1_from_logger.append(iteration.x1Value)
    X2_from_logger.append(iteration.x2Value)
    Z_from_logger.append(iteration.yValue)

In [8]:
constraints = logger_box.getConstraints()
data_of_constraints = []

for constraint in constraints:
    
    #elements = np.array([[0,0]])
    #matrix_x1_x2 = Matrix(1, 2, elements)
    #print constraint.value_at(matrix_x1_x2)
                         
    data_of_current_constraint = []
    #X1_of_constraint_before_meshgrid = []
    #X2_of_constraint_before_meshgrid = []
    Z_of_constraint = []
    X1_of_constraint = []
    X2_of_constraint = []
    for x2 in X2_for_graph_before_meshgrid:
        Z = []
        X1_of_constraint_before_meshgrid = []
        X2_of_constraint_before_meshgrid = []
        #print constraint.value_at(matrix_x1_x2)
        for x1 in X1_for_graph_before_meshgrid:
            elements = np.array([[x1, x2]])
            matrix_x1_x2 = Matrix(1, 2, elements)
            #print "JEDAN"
            #print constraint.value_at(matrix_x1_x2)
            if (constraint.is_satisfied(matrix_x1_x2) is True):
                Z.append(np.nan)
                #print "IF"
                #print constraint.value_at(matrix_x1_x2)
            else:
                #print "else"
                Z.append(constraint.value_at(matrix_x1_x2))
                #print constraint.value_at(matrix_x1_x2)
            #print "DVA"
            #print constraint.value_at(matrix_x1_x2)
            #print "\n"
        Z_of_constraint.append(Z)
    #print Z_of_constraint
    #X1_of_constraint, X2_of_constraint = np.meshgrid(X1_of_constraint_before_meshgrid, X2_of_constraint_before_meshgrid)
    X1_of_constraint, X2_of_constraint = np.meshgrid(X1_for_graph_before_meshgrid, X2_for_graph_before_meshgrid)
    data_of_current_constraint.append(X1_of_constraint)
    data_of_current_constraint.append(X2_of_constraint)
    data_of_current_constraint.append(Z_of_constraint)
    
    data_of_constraints.append(data_of_current_constraint)

In [9]:
w = widgets.IntSlider(min=0, max=playMaxOfInterval - 1, step=1, value=0)
play = widgets.Play(
    value=0,
    min=0,
    max=playMaxOfInterval,
    step=1,
    description="Press play",
    disabled=False
)
nextButton = widgets.Button(description="Next")
previousButton = widgets.Button(description="Previous")

def on_nextButton_clicked(x):
    if (play.value < play.max):
        play.value += 1

def on_previousButton_clicked(x):
    if (play.value > 0):
        play.value -= 1

nextButton.on_click(on_nextButton_clicked)
previousButton.on_click(on_previousButton_clicked)
widgets.jslink((play, 'value'), (w, 'value'))

In [10]:
#region Define function for drawing the whole graph and a single point from the logger
def f(iteration_number, cmap):
    plt.clf()
    plt.close('all')
    #plt.figure(iteration)
    #fig = plt.figure()
    #fig = plt.figure(iteration_number)
    plt.figure(iteration_number)
    ax = plt.axes(projection='3d')
    #ax.contour3D(X1_for_graph, X2_for_graph, Z_for_graph, 50, cmap='Accent')

    # Plot fixed graph
    ax.contour3D(X1_for_graph, X2_for_graph, Z_for_graph, 50, cmap=cmap)
    # plt.plot([-1.9], [2.0], 'b')
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_zlabel('z')

    # Plot single point depending on the iteration
    ax.plot([X1_from_logger[iteration_number]], [X2_from_logger[iteration_number]], [Z_from_logger[iteration_number]],
            markerfacecolor='k', markeredgecolor='k', marker='o', markersize=5, alpha=1)
    #ax.scatter(X1_from_logger, X2_from_logger, Z_from_logger, c='g', marker='^')
    plt.show()
#endregion

In [11]:
#region Define cmap choices
cmap_choices = {
    'Accent': 'Accent',
    'Accent_r': 'Accent_r'}#,

'''
    'Blues': 'Blues',
    'Blues_r': 'Blues_r',
    'BrBG' : 'BrBG',
    'BrBG_r' : 'BrBG_r',
    'BuGn' : 'BuGn',
    'BuGn_r' : 'BuGn_r'
}'''
#endregion

"\n    'Blues': 'Blues',\n    'Blues_r': 'Blues_r',\n    'BrBG' : 'BrBG',\n    'BrBG_r' : 'BrBG_r',\n    'BuGn' : 'BuGn',\n    'BuGn_r' : 'BuGn_r'\n}"

In [12]:
#region Call the function interactively
interact(f, iteration_number=w, cmap = cmap_choices)
#endregion
#region Display remaining widgets
display(play)
display(previousButton)
display(nextButton)
#endregion